# N-Springed Pendulum
In this notebook we'll be solving the equations of motion for an n-springed pendulum which is impossible to do by hand. Based on [lukepolson's work](https://github.com/lukepolson/youtube_channel/blob/main/Python%20Metaphysics%20Series/vid7.ipynb).

## Background
Diagram of a double springed pendulum:

<img src="double_springed_pendulum.svg" alt="diagram" width="400"/>

The variables involved are:
- Time $t$
- Gravitational acceleration $g$
- Mass of pendulum bobs $m_n$
- Spring constants $k_n$
- Natural lengths of springs (when no massed is attached) $l_n$
- Stretched lengths of the springs $r_n$
- Angles that the spring makes with the horizontal plane $\theta_n$

The position of bob $j$, in cartesian coordinates are related to the lengths $l_j$ and $r_j$ and angle $\theta_j$ with the following formulas:

$$\displaystyle x_j= \sum_{n=1}^j (l_n+r_n)cos\theta_n $$
$$\displaystyle y_j= \sum_{n=1}^j -(l_n+r_n)sin\theta_n $$


## Import libraries

In [13]:
import numpy as np
import sympy as smp
import scipy.integrate
import matplotlib.pyplot as plt
import matplotlib.animation

## Define variables

### Set $n$
The number of springs/bobs

In [14]:
n = 2

### Define time, gravitational constant, masses, spring constants, natural lengths, angles and stretched lengths

In [15]:
# Define time and gravitational constant
t, g = smp.symbols('t g')

# Define lists of masses, spring constants and natural lengths
m, k, l = smp.symbols((f'm_:{n}', f'k_:{n}', f'l_:{n}'))

# Define lists of theta's and r's that are functions of time
the = smp.symbols(rf'\theta_:{n}', cls=smp.Function)
the = [the_n(t) for the_n in the]
r = smp.symbols(f'r_:{n}', cls=smp.Function)
r = [r_n(t) for r_n in r]

### Check that the symbols and functions have been correctly defined

In [16]:
r[0]

r_0(t)

### Define the first and second derivatives of $\theta$ and $r$

In [17]:
the_d = [smp.diff(the_n, t) for the_n in the]
the_dd = [smp.diff(the_d_n, t) for the_d_n in the_d]

r_d = [smp.diff(r_n, t) for r_n in r]
r_dd = [smp.diff(r_d_n, t) for r_d_n in r_d]

### Verify definitions

In [18]:
the_dd[0]

Derivative(\theta_0(t), (t, 2))

In [19]:
r_dd[0]

Derivative(r_0(t), (t, 2))

### Define cartesian coordinates of each bob $(x_j,y_j)$ 
where, $x_j$ and $y_j$ are functions of $\theta_{0 \leq i \leq j}$, $r_{0 \leq i \leq j}$ and $l_{0 \leq i \leq j}$


In [20]:
x = list(smp.symbols(f'x_:{n}', cls=smp.Function))
x = [x_j(*the[:j+1], *r[:j+1], *l[:j+1]) for j, x_j in enumerate(x)]
y = list(smp.symbols(f'y_:{n}', cls=smp.Function))
y = [y_j(*the[:j+1], *r[:j+1], *l[:j+1]) for j, y_j in enumerate(y)]

In [21]:
y[0]

y_0(\theta_0(t), r_0(t), l_0)

### Define the equations of the functions
$$\displaystyle x_j= \sum_{i=0}^j (l_i+r_i)cos\theta_i $$
$$\displaystyle y_j= \sum_{i=0}^j -(l_i+r_i)sin\theta_i $$

In [22]:
for j in range(n):
    x[j] = sum([(l[i]+r[i])*smp.cos(the[i]) for i in range(j+1)])
    x[j] = smp.simplify(x[j])
    y[j] = sum([-(l[i]+r[i])*smp.sin(the[i]) for i in range(j+1)])
    y[j] = smp.simplify(y[j])

### Verify equations

In [23]:
y[0]

(-l_0 - r_0(t))*sin(\theta_0(t))

In [24]:
y[1]

-(l_0 + r_0(t))*sin(\theta_0(t)) - (l_1 + r_1(t))*sin(\theta_1(t))

### Define the Lagrangian $L=T-V$ where,


Total kinetic energy: 
$$
T= \displaystyle \sum_{i=0}^{n-1} \frac{1}{2}m_i(\dot{x}^2_i+\dot{y}^2_i)
$$

Total potential energy: 
$$
V = \displaystyle \sum_{i=0}^{n-1} (
        \overbrace{m_i g y_i}^
            {\substack{\text{gravitational} \\ \text{potential} \\ \text{energy}}} +
        \overbrace{\frac{1}{2}k_i r_i^2}^
            {\substack{\text{elastic} \\ \text{potential} \\ \text{energy}}}
    )
$$

In [25]:
T = sum([1/2 * m[i] * (smp.diff(x[i], t)**2 + smp.diff(y[i], t)**2) for i in range(n)])
V = sum([m[i] * g * y[i] + 1/2 * k[i] * r[i]**2 for i in range(n)])
L = T - V

### Verify

In [26]:
T

0.5*m_0*(((-l_0 - r_0(t))*cos(\theta_0(t))*Derivative(\theta_0(t), t) - sin(\theta_0(t))*Derivative(r_0(t), t))**2 + (-(l_0 + r_0(t))*sin(\theta_0(t))*Derivative(\theta_0(t), t) + cos(\theta_0(t))*Derivative(r_0(t), t))**2) + 0.5*m_1*(((-l_0 - r_0(t))*cos(\theta_0(t))*Derivative(\theta_0(t), t) + (-l_1 - r_1(t))*cos(\theta_1(t))*Derivative(\theta_1(t), t) - sin(\theta_0(t))*Derivative(r_0(t), t) - sin(\theta_1(t))*Derivative(r_1(t), t))**2 + (-(l_0 + r_0(t))*sin(\theta_0(t))*Derivative(\theta_0(t), t) - (l_1 + r_1(t))*sin(\theta_1(t))*Derivative(\theta_1(t), t) + cos(\theta_0(t))*Derivative(r_0(t), t) + cos(\theta_1(t))*Derivative(r_1(t), t))**2)

In [27]:
V

g*m_0*(-l_0 - r_0(t))*sin(\theta_0(t)) + g*m_1*(-(l_0 + r_0(t))*sin(\theta_0(t)) - (l_1 + r_1(t))*sin(\theta_1(t))) + 0.5*k_0*r_0(t)**2 + 0.5*k_1*r_1(t)**2

## Compute Lagrange's Equations

$$\frac{\partial L}{\partial z}-\frac{d}{dt}\frac{\partial L}{\partial \dot{z}}=0$$

where $z$ is each of $\{\theta_0, \dots, \theta_{n-1}, r_0, \dots, r_{n-1}\}$

In [32]:
args = [*the, *r]
args_d = [*the_d, *r_d]

LE = [(smp.diff(L, arg) - smp.diff(smp.diff(L, args_d[i]), t)).simplify() for i, arg in enumerate(args)]

In [33]:

print(f'Number of Lagrange equations = {len(LE)}')
LE[0]

Number of Lagrange equations = 4


1.0*g*l_0*m_0*cos(\theta_0(t)) + 1.0*g*l_0*m_1*cos(\theta_0(t)) + 1.0*g*m_0*r_0(t)*cos(\theta_0(t)) + 1.0*g*m_1*r_0(t)*cos(\theta_0(t)) - 1.0*l_0**2*m_0*Derivative(\theta_0(t), (t, 2)) - 1.0*l_0**2*m_1*Derivative(\theta_0(t), (t, 2)) - 1.0*l_0*l_1*m_1*sin(\theta_0(t) - \theta_1(t))*Derivative(\theta_1(t), t)**2 - 1.0*l_0*l_1*m_1*cos(\theta_0(t) - \theta_1(t))*Derivative(\theta_1(t), (t, 2)) - 2.0*l_0*m_0*r_0(t)*Derivative(\theta_0(t), (t, 2)) - 2.0*l_0*m_0*Derivative(\theta_0(t), t)*Derivative(r_0(t), t) - 2.0*l_0*m_1*r_0(t)*Derivative(\theta_0(t), (t, 2)) - 1.0*l_0*m_1*r_1(t)*sin(\theta_0(t) - \theta_1(t))*Derivative(\theta_1(t), t)**2 - 1.0*l_0*m_1*r_1(t)*cos(\theta_0(t) - \theta_1(t))*Derivative(\theta_1(t), (t, 2)) + 1.0*l_0*m_1*sin(\theta_0(t) - \theta_1(t))*Derivative(r_1(t), (t, 2)) - 2.0*l_0*m_1*cos(\theta_0(t) - \theta_1(t))*Derivative(\theta_1(t), t)*Derivative(r_1(t), t) - 2.0*l_0*m_1*Derivative(\theta_0(t), t)*Derivative(r_0(t), t) - 1.0*l_1*m_1*r_0(t)*sin(\theta_0(t) - \th

## Solve Lagrange Equations for $\frac{d^2 z}{dt^2}$
using an ODE solver,
where $z$ is each of the free variables $\{\theta_0, \dots, \theta_{n-1}, r_0, \dots, r_{n-1}\}$.

This will give us two equations for each free variable,
$$\displaystyle \frac{dz}{dt} = v_z$$
$$\displaystyle \frac{dv_z}{dt} = \text{what we solved for}$$

This turns our system of second order ordinary differential equations into systems of first order differential equations.

In [34]:
args_dd = (*the_dd, *r_dd)
sols = smp.solve(LE, args_dd, simplify=False, rational=False)

## Create numerical functions to get cartesian coordinates

In [ ]:
x_f = [smp.lambdify(args=[*the[:j+1], *r[:j+1], *l[:j+1]], expr=x_j) for j, x_j in enumerate(x)]
y_f = [smp.lambdify(args=[*the[:j+1], *r[:j+1], *l[:j+1]], expr=y_j) for j, y_j in enumerate(y)]

Sanity check for numerical functions for $(x_0,y_0)$ with values
- $l_0=1$
- $r_0=1$
- $0 \leq \theta_1 \leq \pi$

Should see a semicircle with radius 2 which is the path traced out by the first bob

In [ ]:
temp_theta = np.linspace(0,np.pi,100)
plt.plot(x_f[0](temp_theta,1,1), y_f[0](temp_theta,1,1), '-r')
plt.axis('equal')
plt.grid(True)